In [ ]:
import yt_dlp
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, widgets, Layout
import browser_cookie3
import subprocess
import http.cookiejar
import json
import os
import io
import sys
import logging
import ffmpeg
import signal
import atexit
from logging.handlers import RotatingFileHandler
import time
import threading
from functools import wraps
from urllib.error import URLError

# Set up logging
logging.basicConfig(filename='youtube_downloader.log', level=logging.DEBUG, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Global flag for graceful shutdown
shutdown_flag = False

def select_format(formats, quality):
    if quality == 'best':
        return 'bestvideo+bestaudio/best'
    elif quality == 'worst':
        return 'worstvideo+worstaudio/worst'
    else:
        requested_height = int(quality[:-1])  # Remove 'p' and convert to int
        # Find the best format that doesn't exceed the requested height
        best_format = None
        for f in formats:
            if f.get('height') and f['height'] <= requested_height:
                if not best_format or f['height'] > best_format['height']:
                    best_format = f
        return best_format['format_id'] if best_format else 'best'

class CleanupThread(threading.Thread):
    def __init__(self, cleanup_interval=3600):
        threading.Thread.__init__(self)
        self.cleanup_interval = cleanup_interval
        self.daemon = True
        self.shutdown_event = threading.Event()

    def run(self):
        while not self.shutdown_event.is_set():
            self.shutdown_event.wait(self.cleanup_interval)
            if not self.shutdown_event.is_set():
                cleanup()

    def stop(self):
        self.shutdown_event.set()

def retry(exceptions, tries=4, delay=3, backoff=2):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    print(f"{str(e)}, Retrying in {mdelay} seconds...")
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return func(*args, **kwargs)
        return wrapper
    return decorator

# Initialize the cleanup thread
cleanup_thread = CleanupThread()
cleanup_thread.start()

def cleanup():
    global YOUTUBE_COOKIES, LOGINS, LOG_OUTPUT
    YOUTUBE_COOKIES = None
    LOGINS = {}
    LOG_OUTPUT.clear_output()
    logging.info("Cleanup completed")

# Make sure to call cleanup when the script exits
import atexit

def exit_handler():
    print("Exiting application...")
    if cleanup_thread:
        cleanup_thread.stop()
        cleanup_thread.join(timeout=5)
    cleanup()
    print("Cleanup finished, exiting now.")


atexit.register(exit_handler)

def signal_handler(signum, frame):
    print("Received interrupt, shutting down gracefully...")
    if cleanup_thread:
        cleanup_thread.stop()

# Set up signal handlers
signal.signal(signal.SIGINT, signal_handler)
signal.signal(signal.SIGTERM, signal_handler)

# Register the exit handler
atexit.register(exit_handler)

def init_cleanup_thread():
    global cleanup_thread
    cleanup_thread = CleanupThread()
    cleanup_thread.start()

# Initialize the cleanup thread when starting the application
init_cleanup_thread()

# Global variables
shutdown_event = threading.Event()
YOUTUBE_COOKIES = None
LOGINS = {}
LOG_OUTPUT = widgets.Output()

# Set up rotating log handler
log_handler = RotatingFileHandler('youtube_downloader.log', maxBytes=1024*1024, backupCount=5)
logging.basicConfig(handlers=[log_handler], level=logging.DEBUG, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Register cleanup function to run on exit
atexit.register(cleanup)

# Periodic cleanup
def periodic_cleanup():
    cleanup()
    # Schedule next cleanup

# Start periodic cleanup
from threading import Timer
Timer(3600, periodic_cleanup).start()

# Custom stream to capture print statements
class LogStream(io.StringIO):
    def write(self, text):
        super().write(text)
        LOG_OUTPUT.append_stdout(text)
        logging.info(text.strip())  # Log to file

# Redirect stdout to our custom stream
sys.stdout = LogStream()

def load_logins():
    global LOGINS
    if os.path.exists('youtube_logins.json'):
        with open('youtube_logins.json', 'r') as f:
            LOGINS = json.load(f)

def save_logins():
    with open('youtube_logins.json', 'w') as f:
        json.dump(LOGINS, f)

def login_youtube(refresh_callback):
    global YOUTUBE_COOKIES, LOGINS
    
    load_logins()
    
    browser_options = ['chrome', 'firefox', 'safari', 'brave', 'edge']
    browser_dropdown = widgets.Dropdown(options=browser_options, description='Browser:')
    login_name = widgets.Text(description='Login Name:')
    login_button = widgets.Button(description='Get Cookies')
    output = widgets.Output()

    def on_button_clicked(b):
        global YOUTUBE_COOKIES
        with output:
            output.clear_output()
            print(f"Fetching cookies from {browser_dropdown.value}...")
            try:
                if browser_dropdown.value == 'chrome':
                    YOUTUBE_COOKIES = browser_cookie3.chrome(domain_name='.youtube.com')
                elif browser_dropdown.value == 'firefox':
                    YOUTUBE_COOKIES = browser_cookie3.firefox(domain_name='.youtube.com')
                elif browser_dropdown.value == 'safari':
                    YOUTUBE_COOKIES = browser_cookie3.safari(domain_name='.youtube.com')
                elif browser_dropdown.value == 'brave':
                    YOUTUBE_COOKIES = browser_cookie3.brave(domain_name='.youtube.com')
                elif browser_dropdown.value == 'edge':
                    YOUTUBE_COOKIES = browser_cookie3.edge(domain_name='.youtube.com')
                
                # Save cookies in Netscape format
                cookie_file = f'youtube_cookies_{login_name.value}.txt'
                with open(cookie_file, 'w') as f:
                    f.write("# Netscape HTTP Cookie File\n")
                    for cookie in YOUTUBE_COOKIES:
                        f.write(f"{cookie.domain}\tTRUE\t{cookie.path}\t"
                                f"{'FALSE' if cookie.expires is None else 'TRUE'}\t"
                                f"{cookie.expires if cookie.expires is not None else 0}\t"
                                f"{cookie.name}\t{cookie.value}\n")
                
                LOGINS[login_name.value] = cookie_file
                save_logins()
                
                print(f"Cookies fetched and saved successfully for {login_name.value}!")
                print("Login process completed. You can now use these cookies for downloads.")
                
                # Call the refresh callback to update the Manage Logins tab
                refresh_callback()
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                YOUTUBE_COOKIES = None

    login_button.on_click(on_button_clicked)
    return widgets.VBox([browser_dropdown, login_name, login_button, output])

def download_video(url, quality='best', login_name=None):
    global shutdown_flag
    output = widgets.Output()

    with output:
        try:
            ydl_opts = {
                'format': 'bestvideo+bestaudio/best',
                'outtmpl': '%(title)s.%(ext)s',
                'writesubtitles': True,
                'allsubtitles': True,
                'subtitlesformat': 'srt',
                'continuedl': True,
                'retries': 10,
                'fragment_retries': 10,
                'skip_unavailable_fragments': True,
                'ignoreerrors': False,
                'verbose': True,
            }

            if login_name and login_name in LOGINS:
                ydl_opts['cookiefile'] = LOGINS[login_name]

            def progress_hook(d):
                if shutdown_flag:
                    raise Exception("Interrupted by user")
                if d['status'] == 'finished':
                    print(f"Done downloading {d['filename']}")
                elif d['status'] == 'downloading':
                    print(f"Downloading {d['filename']}: {d['_percent_str']} complete")

            ydl_opts['progress_hooks'] = [progress_hook]

            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(url, download=False)

                if 'entries' in info:  # It's a playlist
                    print(f"Playlist detected. Found {len(info['entries'])} videos.")
                    for entry in info['entries']:
                        if shutdown_flag:
                            break
                        try:
                            if quality != 'best' and 'formats' in entry:
                                selected_format = select_format(entry['formats'], quality)
                                ydl_opts['format'] = selected_format
                            ydl.download([entry['webpage_url']])
                            time.sleep(1)  # Small delay between downloads
                        except Exception as e:
                            print(f"Error downloading {entry.get('title', 'unknown')}: {str(e)}")
                            logging.exception(f"Error downloading video: {entry.get('webpage_url', 'unknown URL')}")
                else:  # It's a single video
                    if quality != 'best' and 'formats' in info:
                        selected_format = select_format(info['formats'], quality)
                        ydl_opts['format'] = selected_format
                    # ydl.download([url])
                    download_with_retry(ydl,url)

                if not shutdown_flag:
                    print("Download(s) completed successfully!")
                else:
                    print("Download(s) interrupted by user.")
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")
            logging.exception("Unexpected error in download_video")

    return output

def check_ffmpeg():
    try:
        subprocess.run(["ffmpeg", "-version"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except FileNotFoundError:
        return False

def is_vr_video(info):
    # Check multiple indicators for VR/360 content
    indicators = [
        '360' in info.get('title', '').lower(),
        '360' in info.get('description', '').lower(),
        'vr' in info.get('title', '').lower(),
        'vr' in info.get('description', '').lower(),
        any('360' in fmt.get('format', '').lower() for fmt in info['formats']),
        any('vr' in fmt.get('format', '').lower() for fmt in info['formats']),
        info.get('tags') and any('360' in tag.lower() or 'vr' in tag.lower() for tag in info['tags']),
        info.get('stereoscopic') == '3d',
        info.get('projection') in ['equirectangular', 'cubemap'],
    ]
    return any(indicators)

def download_vr_video(url, login_name=None, force_vr=False):
    output = widgets.Output()
    
    with output:
        ydl_opts = {
            'format': 'bestvideo[height<=2160][ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
            'outtmpl': '%(title)s.%(ext)s',
            'writesubtitles': True,
        }
        
        if login_name and login_name in LOGINS:
            ydl_opts['cookiefile'] = LOGINS[login_name]

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            try:
                info = ydl.extract_info(url, download=False)
                
                is_vr = is_vr_video(info)
                if not is_vr and not force_vr:
                    print("This video does not appear to be a VR/360 video.")
                    user_input = input("Do you want to proceed with VR download anyway? (y/n): ")
                    if user_input.lower() != 'y':
                        print("VR download cancelled.")
                        return

                print("Downloading video...")
                # ydl.download([url])
                download_with_retry(ydl,url)

                # Get the filename of the downloaded video
                filename = ydl.prepare_filename(info)

                # Convert the video to a spatial video format compatible with Quest 3 and iOS
                output_filename = f"{os.path.splitext(filename)[0]}_spatial.mp4"
                
                print("Converting video to spatial video format...")
                try:
                    # Prepare FFmpeg command
                    input_stream = ffmpeg.input(filename)
                    
                    # Video stream
                    video_stream = (
                        input_stream.video
                        .filter('scale', 3840, 1920)  # 4K resolution for 180-degree video
                        .filter('fps', fps=60)  # Increase to 60fps for smoother VR experience
                    )
                    
                    # Audio stream
                    audio_stream = input_stream.audio
                    
                    # Output with specific encoding parameters and metadata
                    output = ffmpeg.output(
                        video_stream, 
                        audio_stream, 
                        output_filename,
                        vcodec='libx264',
                        preset='medium',  # Balance between compression and encoding speed
                        crf=18,  # Lower CRF for higher quality (range 0-51, lower is better quality)
                        profile='high',
                        level='5.1',  # Specify H.264 level
                        acodec='aac',
                        audio_bitrate='192k',  # Increased audio bitrate
                        **{
                            'metadata:g:0': 'spatial-audio=true',
                            'metadata:g:1': 'stereo-mode=monoscopic',
                            'metadata:g:2': 'projection-type=equirectangular',
                            'metadata:g:3': 'field-of-view=180',
                            'metadata:g:4': 'spherical=true',
                            'metadata:g:5': 'spatial=1',
                            'metadata:g:6': 'pose-yaw=0',
                            'metadata:g:7': 'pose-pitch=0',
                            'metadata:g:8': 'pose-roll=0',
                        }
                    ).global_args('-movflags', 'faststart')  # Optimize for web streaming
                    
                    # Run FFmpeg command
                    ffmpeg.run(output, capture_stdout=True, capture_stderr=True)
                    
                    print(f"Video conversion completed successfully.")
                    print(f"Spatial video saved as: {output_filename}")
                    
                    # Get file size
                    file_size = os.path.getsize(output_filename) / (1024 * 1024)  # Size in MB
                    print(f"File size: {file_size:.2f} MB")
                    
                    # Remove the original file
                    os.remove(filename)
                    print(f"Original downloaded file removed.")
                except ffmpeg.Error as e:
                    print(f"An error occurred during FFmpeg conversion: {e.stderr.decode()}")
                    logging.error(f"FFmpeg conversion error: {e.stderr.decode()}")
                
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                logging.error(f"Error in download_vr_video: {str(e)}", exc_info=True)

def interactive_download():
    load_logins()

    url_widget = widgets.Textarea(
        description='URL:',
        placeholder='Paste your YouTube URL here (video, playlist, or channel videos)',
        layout=Layout(width='50%', height='100px')
    )
    quality_widget = widgets.Dropdown(
        options=['best', 'vr', '8k', '4k', '1440p', '1080p', '720p', '480p', '360p', '240p', '144p', 'worst'],
        value='best',
        description='Quality:',
    )
    login_dropdown = widgets.Dropdown(
        options=['No login'] + list(LOGINS.keys()),
        description='Login:',
        disabled=False,
    )
    download_button = widgets.Button(description='Download')
    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            output.clear_output()
            selected_login = login_dropdown.value if login_dropdown.value != 'No login' else None
            download_video(url_widget.value.strip(), quality_widget.value, selected_login)

    download_button.on_click(on_button_clicked)
    
    # Save URL when the text area changes
    def save_url(change):
        with open('last_url.txt', 'w') as f:
            f.write(change['new'])
    
    url_widget.observe(save_url, names='value')
    
    # Load last URL if it exists
    if os.path.exists('last_url.txt'):
        with open('last_url.txt', 'r') as f:
            url_widget.value = f.read()

    return widgets.VBox([url_widget, quality_widget, login_dropdown, download_button, output])

def manage_logins():
    load_logins()
    
    login_list = widgets.Select(
        options=list(LOGINS.keys()),
        description='Saved Logins:',
        disabled=False
    )
    delete_button = widgets.Button(description='Delete Selected Login')
    info_area = widgets.Textarea(
        description='Login Info:',
        disabled=True,
        layout=Layout(width='50%', height='100px')
    )
    output = widgets.Output()

    def update_info(change):
        if login_list.value:
            cookie_file = LOGINS[login_list.value]
            info_area.value = f"Login: {login_list.value}\nCookie file: {cookie_file}"
        else:
            info_area.value = ""

    login_list.observe(update_info, names='value')

    def on_delete_clicked(b):
        with output:
            output.clear_output()
            if login_list.value:
                del LOGINS[login_list.value]
                os.remove(f'youtube_cookies_{login_list.value}.txt')
                save_logins()
                print(f"Deleted login: {login_list.value}")
                login_list.options = list(LOGINS.keys())
                info_area.value = ""
            else:
                print("No login selected.")

    delete_button.on_click(on_delete_clicked)
    return widgets.VBox([login_list, info_area, delete_button, output])

def refresh_manage_logins(manage_logins_widget):
    load_logins()
    manage_logins_widget.children[0].options = list(LOGINS.keys())

def run_youtube_downloader():
    tab_contents = ['Download Video', 'Manage Logins', 'Add Login', 'Log']
    manage_logins_widget = manage_logins()
    
    def refresh_callback():
        refresh_manage_logins(manage_logins_widget)
    
    children = [
        widgets.VBox([widgets.Label('Download a video:'), interactive_download()]),
        widgets.VBox([widgets.Label('Manage saved logins:'), manage_logins_widget]),
        widgets.VBox([widgets.Label('Add a new login:'), login_youtube(refresh_callback)]),
        widgets.VBox([widgets.Label('Log:'), LOG_OUTPUT])
    ]
    try:
        tab = widgets.Tab()
        tab.children = children
        for i in range(len(children)):
            tab.set_title(i, tab_contents[i])
        return tab
    except Exception as e:
        logging.exception("Error in run_youtube_downloader")
        return widgets.HTML(f"An error occurred: {str(e)}")

@retry(exceptions=(yt_dlp.utils.DownloadError, URLError), tries=3, delay=5, backoff=2)
def download_with_retry(ydl, url):
    ydl.download([url])

if __name__ == "__main__":
    try:
        display(run_youtube_downloader())
    except Exception as e:
        logging.exception("An unexpected error occurred in the main application")
    finally:
        print("Shutting down application...")
        if cleanup_thread:
            cleanup_thread.stop()
            cleanup_thread.join(timeout=5)
        cleanup()
        print("Application shutdown complete.")